<h1> Create TensorFlow model - DNN</h1>

This notebook illustrates:
<ol>
<li> Creating a model using the high-level Estimator API 
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-3f90a9b55ca0127c'
PROJECT = 'qwiklabs-gcp-3f90a9b55ca0127c'
REGION = 'australia-southeast1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Creating gs://qwiklabs-gcp-3f90a9b55ca0127c/...


<h2> Create TensorFlow model using TensorFlow's Estimator API </h2>
<p>
First, write an input_fn to read the data.
<p>

## Lab Task 1
Verify that the headers match your CSV output

In [4]:
import shutil
import numpy as np
import tensorflow as tf
import logging as log

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [26]:
# Determine CSV, label, and key columns
CSV_COLUMNS = 'weight_pounds,is_male,mother_age,plurality,gestation_weeks,key'.split(',')
LABEL_COLUMN = 'weight_pounds'
KEY_COLUMN = 'key'

# Set default values for each CSV column
DEFAULTS = [[0.0], ['null'], [0.0], ['null'], [0.0], ['nokey']]
TRAIN_STEPS = 1000

## Lab Task 2

Fill out the details of the input function below

In [27]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def read_dataset(filename_pattern, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(line_of_text):
      # TODO #1: Use tf.decode_csv to parse the provided line
        columns = tf.decode_csv(line_of_text, DEFAULTS)
      # TODO #2: Make a Python dict.  The keys are the column names, the values are from the parsed data
        features = dict(zip(CSV_COLUMNS, columns))
      # TODO #3: Return a tuple of features, label where features is a Python dict and label a float
        label = features.pop(LABEL_COLUMN)
        # log.INFO('Label Type:', label.dtype)
        # log.INFO('feature columns:', len(features))
        return features, label
    
    # TODO #4: Use tf.gfile.Glob to create list of files that match pattern
    file_list = tf.gfile.Glob(filename_pattern)

    # Create dataset from file list
    dataset = (tf.data.TextLineDataset(file_list)  # Read text file
                 .map(decode_csv))  # Pass in the decode_csv fun, which is applied to Transform each elem
    
    # TODO #5: In training mode, shuffle the dataset and repeat indefinitely
    #                (Look at the API for tf.data.dataset shuffle)
    #          The mode input variable will be tf.estimator.ModeKeys.TRAIN if in training mode
    #          Tell the dataset to provide data in batches of batch_size 
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None #indefinite
        dataset = dataset.shuffle(buffer_size=10 * batch_size)
    else:
        num_epochs = 1 #ends input after this
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    
    # This will now return batches of features, label
    return dataset # not doing the one shot iterator object thing yet
  return _input_fn

## Lab Task 3

Use the TensorFlow feature column API to define appropriate feature columns for your raw features that come from the CSV.

<b> Bonus: </b> Separate your columns into wide columns (categorical, discrete, etc.) and deep columns (numeric, embedding, etc.)

In [28]:
# Define feature columns
# CSV_COLUMNS = 'weight_pounds,is_male,mother_age,plurality,gestation_weeks,key'
# WIP
featcols = [
    tf.feature_column.numeric_column('gestation_weeks'),
    tf.feature_column.numeric_column('mother_age'),
    tf.feature_column.categorical_column_with_vocabulary_list(
        key='plurality',
        vocabulary_list=['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)', 'Multiple(2+)']),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(
        key='is_male',
        vocabulary_list=['True', 'False', 'Unknown']))
]

In [29]:
featcols # check the column types, not sure if the vocab list column will work, or if it needs to be wrapped in an indicator column?


[_NumericColumn(key='gestation_weeks', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='mother_age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _VocabularyListCategoricalColumn(key='plurality', vocabulary_list=('Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)', 'Multiple(2+)'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='is_male', vocabulary_list=('True', 'False', 'Unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [30]:
# confirmed it need to be wrapped in an indicator column or embedding column:
# ValueError: Items of feature_columns must be a _DenseColumn. You can wrap a categorical column with an embedding_column or indicator_column. Given: _VocabularyListCategoricalColumn(key='plurality', vocabulary_list=('Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)', 'Multiple(2+)'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
featcols = [
    tf.feature_column.numeric_column('gestation_weeks'),
    tf.feature_column.numeric_column('mother_age'),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(
        key='plurality',
        vocabulary_list=['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)', 'Multiple(2+)'])),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(
        key='is_male',
        vocabulary_list=['True', 'False', 'Unknown']))
]

In [31]:
# Separate wide(sparse) vs deep(Dense) column definitions
featcols_wide = [
    tf.feature_column.categorical_column_with_vocabulary_list(
        key='plurality',
        vocabulary_list=['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)', 'Multiple(2+)']),
    tf.feature_column.categorical_column_with_vocabulary_list(
        key='is_male',
        vocabulary_list=['True', 'False', 'Unknown'])
]

featcols_dense = [
    tf.feature_column.numeric_column('gestation_weeks'),
    tf.feature_column.numeric_column('mother_age')
]

## Lab Task 4

To predict with the TensorFlow model, we also need a serving input function (we'll use this in a later lab). We will want all the inputs from our user.

Verify and change the column names and types here as appropriate. These should match your CSV_COLUMNS

In [32]:
# Create serving input function to be able to serve predictions later using provided inputs
def serving_input_fn():
    feature_placeholders = {
        'is_male': tf.placeholder(tf.string, [None]),
        'mother_age': tf.placeholder(tf.float32, [None]),
        'plurality': tf.placeholder(tf.string, [None]),
        'gestation_weeks': tf.placeholder(tf.float32, [None])
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

## Lab Task 5

Complete the TODOs in this code:

In [41]:
# Create estimator to train and evaluate
def train_and_evaluate(output_dir):
  EVAL_INTERVAL = 300
  run_config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL,
                                      keep_checkpoint_max = 3)
  # TODO #1: Create your estimator
  estimator = tf.estimator.DNNRegressor(
    feature_columns=featcols,  # list of processed tf feature columns
    hidden_units=[64, 128],
    # Additional estimator options: need to be passed in to estimator
    model_dir = output_dir,
    config = run_config
  )
  train_spec = tf.estimator.TrainSpec(
                       # TODO #2: Call read_dataset passing in the training CSV file and the appropriate mode
                       input_fn = read_dataset('train.csv', tf.estimator.ModeKeys.TRAIN),
                       max_steps = TRAIN_STEPS)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(
                       # TODO #3: Call read_dataset passing in the evaluation CSV file and the appropriate mode
                       input_fn = read_dataset('eval.csv', tf.estimator.ModeKeys.EVAL),
                       steps = None,
                       start_delay_secs = 60, # start evaluating after N seconds
                       throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
                       exporters = exporter)
  # where it all comes together
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Finally, train!

In [42]:
# Run the model
# Make sure data file is in the folder, otherwise will find 'no new checkpoints'.
shutil.rmtree('babyweight_trained', ignore_errors = True) # start fresh each time
tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
train_and_evaluate('babyweight_trained')

INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_master': '', '_keep_checkpoint_max': 3, '_session_config': None, '_num_ps_replicas': 0, '_task_id': 0, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff56c21b978>, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_train_distribute': None, '_num_worker_replicas': 1, '_save_checkpoints_secs': 300, '_tf_random_seed': None, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_evaluation_master': '', '_model_dir': 'babyweight_trained', '_is_chief': True}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 300 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tens

When I ran it, the final lines of the output (above) were:
<pre>
INFO:tensorflow:Saving dict for global step 1000: average_loss = 1.2693067, global_step = 1000, loss = 635.9226
INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-1000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: babyweight_trained/export/exporter/temp-1517899936/saved_model.pb
</pre>
The exporter directory contains the final model and the final RMSE (the average_loss) is 1.2693067

<h2> Monitor and experiment with training </h2>

In [43]:
from google.datalab.ml import TensorBoard
TensorBoard().start('./babyweight_trained')

TensorBoard was started successfully with pid 6851. Click here to access it.

6851

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License